<a href="https://colab.research.google.com/github/greyhound101/gan_segmentation_FE/blob/main/working_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
import zipfile
with zipfile.ZipFile('/content/gdrive/My Drive/segmentation/Training_Batch1.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [3]:
import os
import glob
for i in range(5,28):
    os.remove( '/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/segmentation-'+str(i)+'.nii')
    os.remove( '/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/volume-'+str(i)+'.nii')

In [4]:
pip install medpy

     |████████████████████████████████| 153kB 14.1MB/s 
     |████████████████████████████████| 47.4MB 60kB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754486 sha256=743636917da64ce5b02ea7c3bad86c5865452527b065ec52521623dc438221bd
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy


In [23]:
from medpy.io import load, save
import os
import os.path
import gc
from scipy.ndimage import zoom
import numpy as np
from tqdm import tqdm
try:
  os.mkdir('data')
except:
  pass
for file in tqdm(range(5)):
        shape=4/2
        img, img_header = load('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/volume-'+str(file)+'.nii')
        seg, img_header = load('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/segmentation-'+str(file)+'.nii')
        arr=np.where(seg==2)[-1]
        mn=(arr.min()+arr.max())/2
        
        
        if mn-shape<0:
          start=0
          end=shape*2
        elif mn+shape>img.shape[-1]:
          end=img.shape[-1]
          start=end-shape*2
        else:
          start=int(max(0,(mn-shape)))
          end=int(min(img.shape[-1],int(mn+shape)))
        
        
        img=img[:,:,start:end]
        img[img < -200] = -200
        img[img > 250] = 250
        img+=112
        img/=117
        img = np.array(img, dtype='float32')
        # img = zoom(img, (0.125, 0.125, 1))
        # print ("Saving image "+str(file))
        # np.save( "/content/data/volume-" + str(file) ,zoom(img, (0.125, 0.125, 1)))
        # print(img.shape)
        # del([img])
        # gc.collect()
        seg=seg.astype('float64')
        seg+=0.1645
        seg/=0.023138
        seg = np.array(seg, dtype='float32')
        seg=seg[:,:,start:end]
        # print ("Saving image "+file)
        # seg=zoom(seg, (0.125, 0.125, 1))
        total=np.concatenate([img,seg],-1)
        print(total.shape)
        np.save( "/content/data/total-" + str(file),total)
        del([seg,img,total])
        gc.collect()


 20%|██        | 1/5 [00:00<00:02,  1.52it/s]

(512, 512, 8)



 40%|████      | 2/5 [00:01<00:02,  1.34it/s]

(512, 512, 8)



 60%|██████    | 3/5 [00:06<00:03,  1.89s/it]

(512, 512, 8)



 80%|████████  | 4/5 [00:18<00:04,  4.92s/it]

(512, 512, 8)



100%|██████████| 5/5 [00:37<00:00,  7.56s/it]

(512, 512, 8)


In [24]:
%%writefile ATLAS_dataset.py
import csv
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import os
from torchvision import transforms
from skimage.transform import resize
import nibabel as nib
from skimage import exposure

class ATLASdataset(Dataset):
    def __init__(self,augmentation=True):
        list_path = []
        for i in range(9):
            root = '../ATLAS_R1.1/Site'+str(i+1)
        
            list_img = os.listdir(root)
            for s in range(len(list_img)):
                list_path.append(os.path.join(root,list_img[s]))

        list_path.sort()
        self.augmentation= augmentation
        self.imglist = list_path
        
    def __len__(self):
        return len(self.imglist)

    def __getitem__(self, index):
        path = os.path.join(self.imglist[index],'t01')
        tempimg = nib.load(os.path.join(path,'T1w_p.nii'))
        B = np.flip(tempimg.get_data(),1)
        sp_size = 64
        img = resize(B, (sp_size,sp_size,sp_size), mode='constant')
        img = 1.0*img
        img = (img-np.min(img))/(np.max(img)-np.min(img))

        if self.augmentation:
            random_n = torch.rand(1)
            if random_n[0] > 0.5:
                img = np.flip(img,0)
                
        img = np.ascontiguousarray(img,dtype=np.float32)

        imageout = torch.from_numpy(img).float().view(1,sp_size,sp_size,sp_size)
        imageout = 2*imageout-1

        return imageout 

Overwriting ATLAS_dataset.py


In [25]:
%%writefile BRATS_dataset.py
import csv
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import os
from skimage.transform import resize
from nilearn import surface
import nibabel as nib
from skimage import exposure

class BRATSdataset(Dataset):
    def __init__(self, train=True, imgtype = 'flair', severity='HGG',is_flip=False,augmentation=True):
        self.augmentation = augmentation
        if train:
            self.root = '../Training_brats/' + severity
        else:
            self.root = '../Validation_brats'
        self.imgtype = imgtype
        list_img = os.listdir(self.root)
        list_img.sort()
        self.imglist = list_img
        self.is_flip = is_flip
        
    def __len__(self):
        return len(self.imglist)

    def __getitem__(self, index):

        path = os.path.join(self.root,self.imglist[index])
        
        img = nib.load(os.path.join(path,self.imglist[index]+'_'+self.imgtype+'.nii.gz'))
        gt = nib.load(os.path.join(path,self.imglist[index])+'_'+'seg.nii.gz')

        A = np.zeros((240,240,166))
        G = np.zeros((240,240,166))
        A[:,:,11:] = img.get_data()
        G[:,:,11:] = gt.get_data()
        x=[]
        y=[]
        z=[]
        
        for i in range(240):
            if np.all(A[i,:,:] ==0):
                x.append(i)
            if np.all(A[:,i,:]==0):
                y.append(i)
            if i <155:
                if np.all(A[:,:,i]==0):
                    z.append(i)

        xl,yl,zl = 0,0,0
        xh,yh,zh = 240,240,155
        for xn in x:
            if xn < 120:
                if xn> xl:
                    xl = xn
            else:
                if xn<xh:
                    xh = xn
        for yn in y:
            if yn < 120:
                if yn> yl:
                    yl = yn
            else:
                if yn<yh:
                    yh = yn
        for zn in z:
            if zn < 77:
                if zn> zl:
                    zl = zn
            else:
                if zn<zh:
                    zh = zn

        B = A[xl-10:xh+10,yl-10:yh+10,zl-10:zh+10]
        B = resize(B, (128, 128, 128), mode='constant')
        
        if self.is_flip:
            B = np.swapaxes(B,1,2)
            B = np.flip(B,1)
            B =np.flip(B,2)
        
        sp_size = 64
        img = resize(B, (sp_size,sp_size,sp_size), mode='constant')
        if self.augmentation:
            random_n = torch.rand(1)
            random_i = 0.3*torch.rand(1)[0]+0.7
            if random_n[0] > 0.5:
                img = np.flip(img,0)

        img = 1.0*img
        img = exposure.rescale_intensity(img)
        img = (img-np.min(img))/(np.max(img)-np.min(img))
        img = 2*img-1

        imageout = torch.from_numpy(img).float().view(1,sp_size,sp_size,sp_size)

        return imageout

Overwriting BRATS_dataset.py


In [26]:
pip install nilearn

In [27]:

import csv
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import os
from torchvision import transforms
from skimage.transform import resize
from nilearn import surface
from medpy.io import *
import glob
class ADNIdataset(Dataset):
	def __init__(self, augmentation=False):
		self.augmentation = augmentation
		self.name = []
		for data in glob.glob('/content/data/total-*.npy'):
			self.name.append(data)
		self.name = np.asarray(self.name)

	def __len__(self):
		return len(self.name)

	def __getitem__(self, index):
		path = self.name[index]
		img = np.load(path)

		img = np.flip(img,1)
		img = np.flip(img,2)
		sp_size = 64
		img = resize(img, (sp_size,sp_size,sp_size), mode='constant')
		if self.augmentation:
			random_n = torch.rand(1)
			random_i = 0.3*torch.rand(1)[0]+0.7
			if random_n[0] > 0.5:
				img = np.flip(img,0)
                
			img = img*random_i.data.cpu().numpy()
            
		imageout = torch.from_numpy(img).float().view(1,sp_size,sp_size,sp_size)
		imageout = imageout*2-1

		return imageout

In [9]:

import numpy as np
import torch
import os
from torch import nn
from torch import optim
from torch.nn import functional as F

class Discriminator(nn.Module):
    def __init__(self, channel=512):
        super(Discriminator, self).__init__()        
        self.channel = channel
        n_class = 1
        
        self.conv1 = nn.Conv3d(1, channel//8, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv3d(channel//8, channel//4, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm3d(channel//4)
        self.conv3 = nn.Conv3d(channel//4, channel//2, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm3d(channel//2)
        self.conv4 = nn.Conv3d(channel//2, channel, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm3d(channel)
        
        self.conv5 = nn.Conv3d(channel, n_class, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x, _return_activations=False):
        h1 = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        h2 = F.leaky_relu(self.bn2(self.conv2(h1)), negative_slope=0.2)
        h3 = F.leaky_relu(self.bn3(self.conv3(h2)), negative_slope=0.2)
        h4 = F.leaky_relu(self.bn4(self.conv4(h3)), negative_slope=0.2)
        h5 = self.conv5(h4)
        output = h5

        return output
    

class Generator(nn.Module):
    def __init__(self, noise:int=1000, channel:int=64):
        super(Generator, self).__init__()
        _c = channel
        
        self.noise = noise
        self.fc = nn.Linear(1000,512*4*4*4)
        self.bn1 = nn.BatchNorm3d(_c*8)
        
        self.tp_conv2 = nn.Conv3d(_c*8, _c*4, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(_c*4)
        
        self.tp_conv3 = nn.Conv3d(_c*4, _c*2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm3d(_c*2)
        
        self.tp_conv4 = nn.Conv3d(_c*2, _c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm3d(_c)
        
        self.tp_conv5 = nn.Conv3d(_c, 1, kernel_size=3, stride=1, padding=1, bias=False)
        
    def forward(self, noise):
        noise = noise.view(-1, 1000)
        h = self.fc(noise)
        h = h.view(-1,512,4,4,4)
        h = F.relu(self.bn1(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv2(h)
        h = F.relu(self.bn2(h))
        
        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv3(h)
        h = F.relu(self.bn3(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv4(h)
        h = F.relu(self.bn4(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv5(h)

        h = F.tanh(h)

        return h

In [30]:
%pylab inline
import numpy as np
import torch
import os

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch import autograd
from torch.autograd import Variable
import nibabel as nib
from torch.utils.data.dataset import Dataset
from torch.utils.data import dataloader
from nilearn import plotting
from BRATS_dataset import *
from ATLAS_dataset import *
from Model_WGAN import *


BATCH_SIZE=4
max_epoch = 100
lr = 0.0001
gpu = True
workers = 4

LAMBDA= 10
#setting latent variable sizes
latent_dim = 1000


trainset = ADNIdataset(augmentation=True)
train_loader = torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,
                                          shuffle=True,num_workers=workers)
if Use_BRATS:
    #'flair' or 't2' or 't1ce'
    trainset = BRATSdataset(imgtype='flair')
    train_loader = torch.utils.data.DataLoader(trainset,batch_size = BATCH_SIZE, shuffle=True,
                                               num_workers=workers)
if Use_ATLAS:
    trainset = ATLASdataset(augmentation=True)
    train_loader = torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,
                                          shuffle=True,num_workers=workers)

def inf_train_gen(data_loader):
    while True:
        for _,images in enumerate(data_loader):
            yield images


D = Discriminator().cuda()
G=Generator()
a=torch.load('/content/gdrive/MyDrive/segmentation/WGAN_G(1).pth',map_location='cuda:0')
G.load_state_dict(a)

g_optimizer = optim.Adam(G.parameters(), lr=0.0002)
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)


def calc_gradient_penalty(netD, real_data, fake_data):    
    alpha = torch.rand(real_data.size(0),1,1,1,1)
    alpha = alpha.expand(real_data.size())
    
    alpha = alpha.cuda()

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = interpolates.cuda()
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty


real_y = Variable(torch.ones((BATCH_SIZE, 1)).cuda())
fake_y = Variable(torch.zeros((BATCH_SIZE, 1)).cuda())
loss_f = nn.BCELoss()

d_real_losses = list()
d_fake_losses = list()
d_losses = list()
g_losses = list()
divergences = list()



Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plotting', 'save', 'shape', 'resize', 'Generator', 'load']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from tqdm import tqdm
TOTAL_ITER = 200000
gen_load = inf_train_gen(train_loader)
for iteration in tqdm(range(TOTAL_ITER)):
    ###############################################
    # Train D 
    ###############################################
    for p in D.parameters():  
        p.requires_grad = True 

    real_images = gen_load.__next__()
    if real_images.shape[0]==1:
      real_images = gen_load.__next__()
      
    D.zero_grad()
    real_images = Variable(real_images).cuda()

    _batch_size = real_images.size(0)


    y_real_pred = D(real_images)

    d_real_loss = y_real_pred.mean()
    
    noise = Variable(torch.randn((_batch_size, latent_dim, 1, 1, 1)),volatile=True).cuda()
    fake_images = G(noise)
    y_fake_pred = D(fake_images.detach())

    d_fake_loss = y_fake_pred.mean()

    gradient_penalty = calc_gradient_penalty(D,real_images.data, fake_images.data)
 
    d_loss = - d_real_loss + d_fake_loss +gradient_penalty
    d_loss.backward()
    Wasserstein_D = d_real_loss - d_fake_loss

    d_optimizer.step()

    ###############################################
    # Train G 
    ###############################################
    for p in D.parameters():
        p.requires_grad = False
        
    for iters in range(5):
        G.zero_grad()
        noise = Variable(torch.randn((_batch_size, latent_dim, 1, 1 ,1)).cuda())
        fake_image =G(noise)
        y_fake_g = D(fake_image)

        g_loss = -y_fake_g.mean()

        g_loss.backward()
        g_optimizer.step()

    # ###############################################
    # # Visualization
    # ###############################################
    # if iteration%10 == 0:
    #     d_real_losses.append(d_real_loss.data[0])
    #     d_fake_losses.append(d_fake_loss.data[0])
    #     d_losses.append(d_loss.data[0])
    #     g_losses.append(g_loss.data.cpu().numpy())

    #     print('[{}/{}]'.format(iteration,TOTAL_ITER),
    #           'D: {:<8.3}'.format(d_loss.data[0].cpu().numpy()), 
    #           'D_real: {:<8.3}'.format(d_real_loss.data[0].cpu().numpy()),
    #           'D_fake: {:<8.3}'.format(d_fake_loss.data[0].cpu().numpy()), 
    #           'G: {:<8.3}'.format(g_loss.data[0].cpu().numpy()))

    #     featmask = np.squeeze((0.5*fake_image+0.5)[0].data.cpu().numpy())

    #     featmask = nib.Nifti1Image(featmask,affine = np.eye(4))
    #     plotting.plot_img(featmask,title="FAKE")
    #     plotting.show()
        
    if (iteration+1)%500 ==0:
        torch.save(G.state_dict(),'./checkpoint/G_W_iter'+str(iteration+1)+'.pth')
        torch.save(D.state_dict(),'./checkpoint/D_W_iter'+str(iteration+1)+'.pth')


  0%|          | 0/200000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.func